In [1]:
import glob
from tqdm import tqdm
from google.colab import drive
import cv2
import numpy as np
from time import time
from os import listdir
from random import choice


drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
!pip install face-alignment

In [0]:
import face_alignment


def rotate(image, angle, center = None, scale = 1.0):
    (h, w) = image.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated


def add_mask(frame):

    global masks_raw, color, label, fa

    tho = thos[color]
    if frame is None:
        raise RuntimeError("No source")
    
    height, width, layers = frame.shape
    res_coef = 1
    new_h = int(height / res_coef)
    new_w = int(width / res_coef)

    frame = cv2.resize(frame, (new_w, new_h))

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    mult = .35
    h_border = int(new_h * mult)
    w_border = int(new_w * mult)

    frame = cv2.copyMakeBorder(frame, h_border, h_border, w_border, w_border, cv2.BORDER_CONSTANT)

    faces = fa.get_landmarks(frame)
    if faces is not None:
        for face in faces:
            try:
                # for landmark in face:  # [face[1], face[15], face[8]]:
                #     if list(landmark) in [list(face[1]), list(face[15]), list(face[8]), list(face[30])]:
                #         cv2.circle(frame, (landmark[0], landmark[1]), 2, (255, 255, 0), -1)
                #     else:
                #         cv2.circle(frame, (landmark[0], landmark[1]), 2, (255, 0, 0), -1)

                x1, x2, x3, y1, y2, y3, z1, z2 = int(face[1][0]), int(face[15][0]), int(face[8][0]), int(face[1][1]), \
                                                 int(face[15][1]), int(face[8][1]), int(face[1][2]), int(face[15][2])

                try:
                    # detecting angles
                    yaw = -np.arctan((z1 - z2) / abs(x1 - x2)) / np.pi * 180
                    roll = -np.arctan((y2 - y1) / (x2 - x1)) / np.pi * 180
                    pitch = -np.arctan((face[28][1] - face[1][1]) / (face[28][2] - face[1][2])) / np.pi * 180
                except ZeroDivisionError:
                    continue

                # rotate frame in opposite to face rotation
                rot_frame = rotate(frame, -roll)

                #----------------------------------
                double = fa.get_landmarks(rot_frame)
                face = double[faces.index(face)]
                #----------------------------------
                # print(len(faces), end='\r')

                # roll_rad = roll * np.pi / 180
                # p = rot_frame.shape[1] / 2
                # q = rot_frame.shape[0] / 2
                # x1 = (x1-p)*np.cos(roll_rad) - (y1)

                x1, x2, x3, y1, y2, y3, z1, z2 = int(face[1][0]), int(face[15][0]), int(face[8][0]), int(face[1][1]), \
                                                 int(face[15][1]), int(face[8][1]), int(face[1][2]), int(face[15][2])

                # choosing appropriate mask image
                angle_deltas = [abs(abs(yaw) - op) for op in masks_raw.keys()]
                min_angle_index = angle_deltas.index(min(angle_deltas))

                face_width = int(np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2))
                # face_width -= int(face_width*abs(pitch)*.001) # adjusting mask width with respect to pitch angle

                y1 = int(face[28][1])
                if label == 2:
                    y1 += int(abs(y1 - y3) * .35)

                # flipping if necessary

                if yaw >= 0:
                    mask_raw = masks_raw[list(masks_raw.keys())[min_angle_index]]
                    roi = rot_frame[y1:y3, x2 - face_width:x2]
                else:
                    mask_raw = cv2.flip(masks_raw[list(masks_raw.keys())[min_angle_index]], 1)
                    roi = rot_frame[y1:y3, x1:x1 + face_width]

                size_mask = (face_width, abs(y1 - y3))

                mask_resized = cv2.resize(mask_raw, size_mask)
                gray_mask = cv2.cvtColor(mask_resized, cv2.COLOR_BGR2GRAY)
                _, mask = cv2.threshold(gray_mask, tho, 255, cv2.THRESH_BINARY_INV)
                mask_inv = cv2.bitwise_not(mask)

                try:
                    masked_face = cv2.bitwise_and(mask_resized, mask_resized, mask=mask)
                    masked_frame = cv2.bitwise_and(roi, roi, mask=mask_inv)

                    # rot_frame = rotate(rot_frame, roll)
                    if yaw >= 0:
                        rot_frame[y1:y3, x2 - face_width:x2] = cv2.add(masked_face, masked_frame)
                    else:
                        rot_frame[y1:y3, x1:x1 + face_width] = cv2.add(masked_face, masked_frame)
                    # rot_frame = rotate(rot_frame, -roll)
                except cv2.error as e:
                    continue

                frame = rotate(rot_frame, roll)
            except ValueError as e:
                # print(e)
                continue

    try:
        roll == 1
    except UnboundLocalError:
        roll = 0

    # cv2.putText(frame, str(tho), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1)
    # frame = rotate(frame, roll)

    frame_size = frame.shape
    frame = frame[h_border:frame_size[0] - h_border, w_border:frame_size[1] - w_border]

    return frame

In [10]:
root = '/content/drive/My Drive/new_set/'
listdir(root)

['lfw', 'new', 'masks', 'Untitled']

In [11]:
print('Reading pics')
faces = glob.glob('/content/drive/My Drive/new_set/lfw/*/*.jpg')
import numpy as np
from skimage import io

Reading pics


In [18]:
filenames = [file for file in  listdir(f'{root}masks/blue') if file[-3:] == 'png']
colors = ['blue', 'black', 'green']
print('Reading masks')
masks_raw_all = {c:
                 {int(fname[:-4]): cv2.imread(f'{root}masks/{c}/{fname}', 1) for fname in filenames}
             for c in colors}
thos = {'blue': 220, 'black': 100, 'green': 145} # mask thesholds

proc = []
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, flip_input=True, device='cuda')

for filepath in tqdm(faces):
    fname = filepath.split('/')[-1][:-4]
    roll = np.random.randint(0,3)
    color = choice(colors)
    masks_raw = masks_raw_all[color]
    output = cv2.imread(filepath)
    # no mask
    if roll != 1:
        label = roll
        output = add_mask(output)

    cv2.imwrite(root + 'new/{}/{}.png'.format(roll, fname), output)

Reading masks


  0%|          | 45/13233 [00:26<3:15:37,  1.12it/s]

  0%|          | 49/13233 [00:34<5:27:02,  1.49s/it]

  3%|▎         | 382/13233 [03:25<2:06:06,  1.70it/s]

  4%|▎         | 474/13233 [04:14<2:25:28,  1.46it/s]

...

 95%|█████████▍| 12535/13233 [1:47:39<06:34,  1.77it/s]

 97%|█████████▋| 12878/13233 [1:50:37<02:38,  2.24it/s]

 97%|█████████▋| 12901/13233 [1:50:51<04:54,  1.13it/s]

100%|█████████▉| 13184/13233 [1:53:18<00:33,  1.45it/s]

100%|██████████| 13233/13233 [1:53:42<00:00,  1.94it/s]


## In case of using this with web camera or video source

In [0]:
# -------------
source_types = ['cam', 'video', 'pic']
path_to_video = 'C:/projects/dyno/imgs/video/polina.mp4'
path_to_pics  = 'C:/projects/dyno/imgs/me'
source_type = source_types[1]
colors = ['blue', 'black', 'green']
color = 'blue'
label = 0  # 0 - mask, 1 - without, 2 - wrong
#--------------

filenames = [file for file in  listdir(f'imgs/masks/{color}') if file[-3:] == 'png']
masks_raw_all = {c:
                 {int(fname[:-4]): cv2.imread(f'imgs/masks/{color}/{fname}', 1) for fname in filenames}
             for c in colors}
masks_raw = masks_raw_all[color]

if source_type == 'cam':
    cap = cv2.VideoCapture(0)
elif source_type == 'video':
    cap = cv2.VideoCapture(path_to_video)

t0 = int(time())

thos = {'blue': 220, 'black': 100, 'green': 145} # mask thesholds

proc = []
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, flip_input=True, device='cuda')


if source_type == 'cam':
    while True:
        _, frame = cap.read()
        cv2.imshow("Frame", add_mask(frame))

        key = cv2.waitKey(1)
        if key == 27:
            cv2.destroyAllWindows()
            cap.release()
            break

elif source_type == 'video':
    i = 0
    path_to_save = '/'.join(path_to_video.split('/')[:-1]) + '/' + path_to_video.split('/')[-1].split('.')[0] + '_OUT.mp4'
    # fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    width = int(cap.get(3))
    height = int(cap.get(4))
    out = cv2.VideoWriter(path_to_save, 0x7634706d, 25.0, (width, height))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    while True:
        i += 1
        print(f'{i}/{frame_count}', end='\tfaces: ')
        ret, frame = cap.read()
        if ret:
            try:
                frame = add_mask(frame)
            except TypeError as e:
                print(e)
                pass
            out.write(frame)
            cv2.imshow('frame', frame)
            key = cv2.waitKey(1)
            if key == 27:
                cv2.destroyAllWindows()
                cap.release()
                out.release()
                break
        else:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

elif source_type == 'pic':
    errs = 0
    path = path_to_pics + '/1/'
    save_path = path_to_pics + '/' + str(label)
    pic_paths = [path + p for p in listdir(path)]
    for pic_path in tqdm(pic_paths):
        try:
            color = choice(colors)
            masks_raw = masks_raw_all[color]
            pic_name = pic_path.split('/')[-1]
            pic = cv2.imread(pic_path)
            cv2.imwrite(save_path+'/'+pic_name, add_mask(pic))
        except ValueError:
            errs += 1
            continue
